# Energy Model - Isle of Eigg

##### Created on: 25.12.2023
##### Last updated on: 17.02.2024
##### Version: 07

##### Developed by: Taha Ahmed Siddiqui (tahaahmedsiddiqui@outlook.com) and Hareem Taha (hareem.nadeem10@gmail.com)
##### GitHub Repository: https://github.com/TahaAhmedSiddiqui/Energy-Modelling---Isle-of-Eigg

##### Use this code for multi-year analysis of Eigg's Energy System and decarbonization scenarios. For the current system's simulation and analysis, refer to the (current scenario model) in the repository.

In [1]:
#importing required packages
import pyomo.environ as pyo
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#initiating the model
#naming the model 'EM' -- stands for Energy Model
EM = pyo.AbstractModel()

#### Sets

In [3]:
#reading the csv's for sets
technology = pd.read_csv('../1. Input/technologies.csv')
fuel = pd.read_csv('../1. Input/fuels.csv')
storage = pd.read_csv('../1. Input/storages.csv')

#creating sets
EM.Technology = pyo.Set(initialize = technology['technology'])
EM.Fuel = pyo.Set(initialize = fuel['fuel'])
EM.Storage = pyo.Set(initialize = storage['storage'])
EM.Hour = pyo.RangeSet(1,8760,1)
EM.Year = pyo.RangeSet(2026,2030,2)

#### Parameters

In [4]:
#preparation for the declaration of parameters
#saving the no. of hours define
n_hour= len(EM.Hour.data())

#Creating the yearly difference multiplier to get the number of years between each timestep
YearlyDifferenceMultiplier = [EM.Year.at(i+1) - EM.Year.at(i) for i in range(1,len(EM.Year))]
YearlyDifferenceMultiplier.append(1)
YearlyDifferenceMultiplier = pd.Series(YearlyDifferenceMultiplier, index=EM.Year, name='value')

In [5]:
#reading the csv's for technology
readin_variablecost = pd.read_csv('../1. Input/variablecost.csv', index_col=[0], sep=',').squeeze('columns')
readin_inputratio = pd.read_csv('../1. Input/inputratio.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_outputratio = pd.read_csv('../1. Input/outputratio.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_emissionratio = pd.read_csv('../1. Input/emissionratio.csv', index_col=[0], sep=',').squeeze('columns')
readin_maxcapacity = pd.read_csv('../1. Input/maxcapacity.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_capacityfactor = pd.read_csv('../1. Input/capacity_factors.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_tagdispatchabletechnology = pd.read_csv('../1. Input/tagdispatchabletechnology.csv', index_col=[0], sep=',').squeeze('columns')
readin_investmentcost = pd.read_csv('../1. Input/investmentcost.csv', index_col=[0], sep=',').squeeze('columns')
readin_residualcapacity = pd.read_csv('../1. Input/residualcapacity.csv', index_col=[0], sep=',').squeeze('columns')
readin_taggridtechnology = pd.read_csv('../1. Input/taggridtechnology.csv', index_col=[0], sep=',').squeeze('columns')
readin_technologylifetime = pd.read_csv('../1. Input/technologylifetime.csv', index_col=[0], sep=',').squeeze('columns')
readin_technologymincapacity = pd.read_csv('../1. Input/technologymincapacity.csv', index_col=[0], sep=',').squeeze('columns')
readin_installedtechnologyyear = pd.read_csv('../1. Input/installedtechnologyyear.csv', index_col = [0], sep=',').squeeze('columns')

#reading the csv's for demand
readin_demand = pd.read_csv('../1. Input/demand.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_demandprofile = pd.read_csv('../1. Input/demand_timeseries.csv', index_col=[0,1], sep=',').squeeze('columns')

#reading the csv's for storages
readin_e2pratio = pd.read_csv('../1. Input/e2pratio.csv', index_col=[0], sep=',').squeeze('columns')
#readin_storagechargeefficiency = pd.read_csv('../1. Input/storagechargeefficiency.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_storagedischargeefficiency = pd.read_csv('../1. Input/storagedischargeefficiency.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_maxstoragecapacity = pd.read_csv('../1. Input/maxstoragecapacity.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_investmentcoststorage = pd.read_csv('../1. Input//investmentcoststorage.csv', index_col=[0], sep=',').squeeze('columns')
readin_residualcapacitystorage = pd.read_csv('../1. Input/residualcapacitystorage.csv', index_col=[0,1], sep=',').squeeze('columns')
readin_storagelifetime = pd.read_csv('../1. Input/storagelifetime.csv', index_col=[0], sep=',').squeeze('columns')
readin_installedstorageyear = pd.read_csv('../1. Input/installedstorageyear.csv', index_col=[0], sep=',').squeeze('columns')

#reading the csv for annual emission budget
readin_annualemissionlimit = pd.read_csv('../1. Input/annualemissionlimit.csv', index_col=[0], sep=',').squeeze('columns')

#reading the csv for final year generator share
readin_generatorshare = pd.read_csv('../1. Input/finalyeardieselgeneratorshare.csv', index_col=[0], sep=',').squeeze('columns')

EM.GeneratorShare = pyo.Param(EM.Technology, default=0, initialize=readin_generatorshare)

#declaring parameters
#for technology
EM.VariableCost = pyo.Param(EM.Technology, default=0, initialize=readin_variablecost)
EM.OutputRatio = pyo.Param(EM.Technology, EM.Fuel, default=0, initialize=readin_outputratio)
EM.InputRatio = pyo.Param(EM.Technology, EM.Fuel, default=0 , initialize=readin_inputratio)
EM.EmissionRatio = pyo.Param(EM.Technology, default=0, initialize=readin_emissionratio)
EM.MaxCapacity = pyo.Param(EM.Year, EM.Technology, default=float('inf'), initialize=readin_maxcapacity)
EM.CapacityFactor = pyo.Param(EM.Technology, EM.Hour, default=0, initialize=readin_capacityfactor, mutable=True)
EM.TagDispatchableTechnology = pyo.Param(EM.Technology, default=1, initialize=readin_tagdispatchabletechnology)
EM.CapitalCost = pyo.Param(EM.Technology, default=0, initialize=readin_investmentcost)
EM.ResidualCapacity = pyo.Param(EM.Technology,default=0,initialize=readin_residualcapacity)
EM.TagGridTechnology = pyo.Param(EM.Technology, default=0,initialize=readin_taggridtechnology)
EM.TechnologyLifetime = pyo.Param(EM.Technology, default=25, initialize=readin_technologylifetime)
EM.TechnologyMinCapacity = pyo.Param(EM.Technology, default = 1, initialize=readin_technologymincapacity)
EM.TechnologyInstallationYear = pyo.Param(EM.Technology, default=2025, initialize=readin_installedtechnologyyear)

#for demand
EM.Demand = pyo.Param(EM.Year, EM.Fuel, default =0, initialize=readin_demand)
EM.DemandProfile = pyo.Param(EM.Fuel, EM.Hour, default=1/len(EM.Hour.data()), initialize=readin_demandprofile)

#for storage
EM.E2PRatio = pyo.Param(EM.Storage, default=0, initialize=readin_e2pratio)
#EM.StorageChargeEfficiency = pyo.Param(EM.Storage, EM.Fuel, default=1, initialize=readin_storagechargeefficiency)
EM.StorageDisChargeEfficiency = pyo.Param(EM.Storage, EM.Fuel, default=1, initialize=readin_storagedischargeefficiency)
EM.MaxStorageCapacity = pyo.Param(EM.Year,EM.Storage, default=float('inf'), initialize=readin_maxstoragecapacity)
EM.InvestmentCostStorage = pyo.Param(EM.Storage, default=0, initialize=readin_investmentcoststorage)
EM.ResidualCapacityStorage = pyo.Param(EM.Storage, EM.Fuel, default=0,initialize=readin_residualcapacitystorage)
EM.StorageLifetime = pyo.Param(EM.Storage, default = 20, initialize= readin_storagelifetime)
EM.StorageInstallationYear = pyo.Param(EM.Storage, default = 2025, initialize= readin_installedstorageyear)

#for emissions
EM.AnnualEmissionLimit = pyo.Param(EM.Year, default=float('inf'), initialize=readin_annualemissionlimit)

#for grid capacity
EM.GridCapacity = pyo.Param(initialize = 725)

#for economics
EM.DiscountRate = pyo.Param(initialize = 0.03)
EM.YearlyDifferenceMultiplier = pyo.Param(EM.Year, initialize=YearlyDifferenceMultiplier)

In [6]:
#preparation for the declaration of variables
#this block of code sets the CapacityFactor of technologies that are dispatchable (like diesel generator) to be always 1
#first, we need to construct our sets in order to be able to loop over them
EM.Technology.construct()
EM.TagDispatchableTechnology.construct()
EM.CapacityFactor.construct()
#then, we can loop over t and h and set the CapacityFactor to always be 1 when TagDispatchableTechnology is non-zero
for t in EM.Technology:
    for h in EM.Hour:
        if EM.TagDispatchableTechnology[t] != 0:
            EM.CapacityFactor[t,h] = 1

#### Variables

In [7]:
#define the variables here
EM.TotalCost = pyo.Var(EM.Technology, domain=pyo.NonNegativeReals)
EM.FuelProductionByTechnology = pyo.Var(EM.Year, EM.Hour, EM.Technology, EM.Fuel, domain=pyo.NonNegativeReals)
EM.NewCapacity = pyo.Var(EM.Year, EM.Technology, domain=pyo.NonNegativeReals)
EM.TotalCapacity = pyo.Var(EM.Year, EM.Technology, domain=pyo.NonNegativeReals)
EM.FuelUseByTechnology = pyo.Var(EM.Year, EM.Hour, EM.Technology, EM.Fuel, domain=pyo.NonNegativeReals)
EM.AnnualTechnologyEmissions = pyo.Var(EM.Year, EM.Technology)
EM.Curtailment = pyo.Var(EM.Year, EM.Hour,EM.Fuel, domain=pyo.NonNegativeReals)
EM.NewStorageEnergyCapacity = pyo.Var(EM.Year, EM.Storage,EM.Fuel, domain=pyo.NonNegativeReals) 
EM.TotalStorageCapacity = pyo.Var(EM.Year, EM.Storage, EM.Fuel, domain=pyo.NonNegativeReals)
EM.StorageCharge = pyo.Var(EM.Year,EM.Hour,EM.Storage,EM.Fuel, domain=pyo.NonNegativeReals) 
EM.StorageDisCharge = pyo.Var(EM.Year, EM.Hour,EM.Storage,EM.Fuel, domain=pyo.NonNegativeReals) 
EM.StorageLevel = pyo.Var(EM.Year, EM.Hour,EM.Storage,EM.Fuel, domain=pyo.NonNegativeReals) 
EM.TotalStorageCost = pyo.Var(EM.Year,EM.Storage, domain=pyo.NonNegativeReals)
EM.SalvageValue = pyo.Var(EM.Year, EM.Technology, domain=pyo.NonNegativeReals)
EM.CapacityUnits = pyo.Var(EM.Year, EM.Technology, domain=pyo.NonNegativeIntegers)
EM.InfeasibleGeneration = pyo.Var(EM.Year,EM.Hour,EM.Fuel, domain=pyo.NonNegativeReals)
EM.TotalCapitalCost = pyo.Var(EM.Year, EM.Technology, domain=pyo.NonNegativeReals)
EM.TotalVariableCost = pyo.Var(EM.Year, EM.Hour, EM.Technology, domain=pyo.NonNegativeReals)

#### Objective Function

In [8]:
#defining the objective function here
#the objective is to minimize the total costs of the system

def obj_function(EM): 
    return (pyo.summation(EM.TotalCost) + pyo.summation(EM.TotalStorageCost)
            -pyo.summation(EM.SalvageValue))+pyo.summation(EM.InfeasibleGeneration)*9999 + pyo.summation(EM.Curtailment)


#creating the objective object with the earlier created objective function
EM.obj = pyo.Objective(rule=obj_function)


#### Constraints

In [9]:
#defining the constraint functions/expressions and create constraint objects
#writing the constraint functions here

#for technology
def demand_adequacy(EM,y,h,f): 
    return (
        sum(EM.FuelProductionByTechnology[y,h,t,f] for t in EM.Technology) 
        + sum(EM.StorageDisCharge[y,h,s,f] for s in EM.Storage if EM.StorageDisChargeEfficiency[s,f]>0) + EM.InfeasibleGeneration[y,h,f] == 
        EM.Demand[y,f]*EM.DemandProfile[f,h] 
        + sum(EM.FuelUseByTechnology[y,h,t,f] for t in EM.Technology) 
        + EM.Curtailment[y,h,f]
        + sum(EM.StorageCharge[y,h,s,f] for s in EM.Storage if EM.StorageDisChargeEfficiency[s,f]>0) 
    )

def productioncost(EM,t): 
    return (
        sum(EM.TotalVariableCost[y,h,t] * EM.YearlyDifferenceMultiplier[y] for y in EM.Year for h in EM.Hour) 
            + sum(EM.TotalCapitalCost[y,t] for y in EM.Year) == EM.TotalCost[t]
    )

def totalcapitalcostfunction(EM,y,t):
    return EM.NewCapacity[y,t] * EM.CapitalCost[t] / (1+EM.DiscountRate)**(y-min(EM.Year)) == EM.TotalCapitalCost[y,t]

def totalvariablecostfunction(EM,y,h,t):
    return sum(EM.FuelProductionByTechnology[y,h,t,f] / (1+EM.DiscountRate)**(y-min(EM.Year))
            * EM.VariableCost[t] for f in EM.Fuel) == EM.TotalVariableCost[y,h,t]

def productionfunction_dispatchable(EM,y,h,t,f):
    if EM.TagDispatchableTechnology[t] > 0:
        return EM.OutputRatio[t,f] * EM.TotalCapacity[y,t] * EM.CapacityFactor[t,h] >= EM.FuelProductionByTechnology[y,h,t,f]
    else:
        return EM.OutputRatio[t,f] * EM.TotalCapacity[y,t] * EM.CapacityFactor[t,h] == EM.FuelProductionByTechnology[y,h,t,f]

def usefunction(EM,y,h,t,f): 
    return EM.InputRatio[t,f] * sum(EM.FuelProductionByTechnology[y,h,t,ff] for ff in EM.Fuel) == EM.FuelUseByTechnology[y,h,t,f]

def maxcapacityconstraint(EM,y,t):
    return EM.TotalCapacity[y,t] <= EM.MaxCapacity[y,t]

def totalcapacityaccounting(EM,y,t):
    if y >= EM.TechnologyInstallationYear[t] + EM.TechnologyLifetime[t]:
        return sum(EM.NewCapacity[yy,t] for yy in EM.Year if (yy <= y and (yy+EM.TechnologyLifetime[t]) >= y)) == EM.TotalCapacity[y,t]
    else:
        return sum(EM.NewCapacity[yy,t] for yy in EM.Year if (yy <= y and (yy+EM.TechnologyLifetime[t]) >= y)) + EM.ResidualCapacity[t] == EM.TotalCapacity[y,t]

def gridcapacity(EM,y,h):
    return (sum(EM.FuelProductionByTechnology[y,h,t,f] for t in EM.Technology for f in EM.Fuel if EM.TagGridTechnology[t] == 1) 
            + sum(EM.StorageDisCharge[y,h,s,f] for s in EM.Storage for f in EM.Fuel if EM.StorageDisChargeEfficiency[s,f]>0)) <= EM.GridCapacity

def capacityunitfunction(EM,y,t):
    return EM.NewCapacity[y,t] == EM.TechnologyMinCapacity[t] * EM.CapacityUnits[y,t]

#for emissions
def technologyemissionfunction(EM,y,t):
    return sum(EM.FuelProductionByTechnology[y,h,t,f] for f in EM.Fuel for h in EM.Hour) * EM.EmissionRatio[t] == EM.AnnualTechnologyEmissions[y,t]

def annualemissionlimitconstraint(EM,y):
    return sum(EM.AnnualTechnologyEmissions[y,t] for t in EM.Technology) <= EM.AnnualEmissionLimit[y]


#for storage
def storagechargefunction(EM,y,s,h,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0:
        return EM.StorageCharge[y,h,s,f] <= EM.TotalStorageCapacity[y,s,f]/EM.E2PRatio[s]
    else:
        return pyo.Constraint.Skip

def storagedischargefunction(EM,y,s,h,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0:
        return EM.StorageDisCharge[y,h,s,f] <= EM.TotalStorageCapacity[y,s,f]/EM.E2PRatio[s]
    else:
        return pyo.Constraint.Skip

def storagelevelfunction(EM,y,s,h,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0 and h>1:
        return (
            EM.StorageLevel[y,h,s,f] == EM.StorageLevel[y,h-1,s,f] + EM.StorageCharge[y,h,s,f] - EM.StorageDisCharge[y,h,s,f]
        )
    else:
        return pyo.Constraint.Skip

def storagelevelstartfunction(EM,y,s,h,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0 and h==1:
        return (
            EM.StorageLevel[y,h,s,f] == 0.5*EM.TotalStorageCapacity[y,s,f] + EM.StorageCharge[y,h,s,f] - EM.StorageDisCharge[y,h,s,f]
        )
    else:
        return pyo.Constraint.Skip

def maxstoragelevelfunction(EM,y,s,h,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0:
        return EM.StorageLevel[y,h,s,f] <= EM.TotalStorageCapacity[y,s,f]
    else:
        return pyo.Constraint.Skip
    
def storagecostfunction(EM,y,s): 
        return (
            EM.TotalStorageCost[y,s] == sum(EM.NewStorageEnergyCapacity[y,s,f]*EM.InvestmentCostStorage[s] / (1+EM.DiscountRate)**(y-min(EM.Year)) for f in EM.Fuel))

def storageannualbalancefunction(EM,y,s,f): 
    if EM.StorageDisChargeEfficiency[s,f] > 0:
        return (
            EM.StorageLevel[y,n_hour,s,f] == 0.5*EM.TotalStorageCapacity[y,s,f]
        )
    else:
        return pyo.Constraint.Skip
    
def totalstoragecapacityaccounting(EM,y,s,f):
    if y >= EM.StorageInstallationYear[s] + EM.StorageLifetime[s]:
        return sum(EM.NewStorageEnergyCapacity[yy,s,f] for yy in EM.Year if (yy<=y and (yy+EM.StorageLifetime[s]) >= y)) == EM.TotalStorageCapacity[y,s,f]
    else:
        return sum(EM.NewStorageEnergyCapacity[yy,s,f] for yy in EM.Year if (yy<=y and (yy+EM.StorageLifetime[s]) >= y)) + EM.ResidualCapacityStorage[s,f] == EM.TotalStorageCapacity[y,s,f]

def storagemaxcapacityconstraint(EM,y,s):
    return sum(EM.TotalStorageCapacity[y,s,f] for f in EM.Fuel) <= EM.MaxStorageCapacity[y,s]

#for economics
def salvagevalue_endoflife(EM,y,t):
    if (EM.TechnologyLifetime[t]+y) < max(EM.Year):
        return (
            EM.SalvageValue[y,t] == 0        
        )
    else:
        return pyo.Constraint.Skip

def salvagevalue_duringlife(EM,y,t):
    if (EM.TechnologyLifetime[t]+y) >= max(EM.Year):
        return (
            EM.SalvageValue[y,t] == 
            EM.NewCapacity[y,t] * EM.CapitalCost[t] / (1+EM.DiscountRate)**(y - min(EM.Year))
            * (1-((max(EM.Year)-y)/EM.TechnologyLifetime[t]))
        )
    else:
        return pyo.Constraint.Skip

#for final year generator share
def maxgeneratorshare(EM,y,h,t):
    if y == 2030:
        return EM.FuelProductionByTechnology[y,h,'DieselGenerator','Power'] <=  EM.Demand[y, 'Power']*EM.DemandProfile['Power',h]*EM.GeneratorShare[t] 
    else:
        return pyo.Constraint.Skip

EM.MaximumGeneratorShare = pyo.Constraint(EM.Year,EM.Hour,EM.Technology,rule=maxgeneratorshare)


#creating the constraint objects

#for technology    
EM.DemandAdequacy = pyo.Constraint(EM.Year,EM.Hour,EM.Fuel, rule=demand_adequacy)
EM.ProductionCost = pyo.Constraint(EM.Technology, rule=productioncost)
EM.ProductionFunction_disp = pyo.Constraint(EM.Year,EM.Hour,EM.Technology, EM.Fuel, rule=productionfunction_dispatchable)
#EM.ProductionFunction_res = pyo.Constraint(EM.Year,EM.Hour,EM.Technology, EM.Fuel, rule=productionfunction_variable)
EM.UseFunction = pyo.Constraint(EM.Year,EM.Hour,EM.Technology, EM.Fuel, rule=usefunction)
EM.TechnologyEmissionFunction = pyo.Constraint(EM.Year,EM.Technology, rule=technologyemissionfunction)
EM.AnnualEmissionLimitConstraint = pyo.Constraint(EM.Year, rule=annualemissionlimitconstraint)
EM.MaxCapacityConstraint = pyo.Constraint(EM.Year,EM.Technology, rule=maxcapacityconstraint)
EM.TotalCapacityAccounting = pyo.Constraint(EM.Year, EM.Technology, rule=totalcapacityaccounting)
EM.CapacityUnitFunction = pyo.Constraint(EM.Year, EM.Technology, rule=capacityunitfunction)
EM.TotalCapitalCostFunction = pyo.Constraint(EM.Year, EM.Technology, rule=totalcapitalcostfunction)
EM.TotalVariableCostFunction = pyo.Constraint(EM.Year, EM.Hour, EM.Technology, rule=totalvariablecostfunction)

#for storage
EM.StorageChargeFunction = pyo.Constraint(EM.Year,EM.Storage,EM.Hour,EM.Fuel, rule=storagechargefunction)
EM.StorageDisChargeFunction = pyo.Constraint(EM.Year,EM.Storage,EM.Hour,EM.Fuel, rule=storagedischargefunction)
EM.StorageLevelFunction = pyo.Constraint(EM.Year,EM.Storage,EM.Hour,EM.Fuel, rule=storagelevelfunction)
EM.StorageLevelStartFunction = pyo.Constraint(EM.Year,EM.Storage,EM.Hour,EM.Fuel, rule=storagelevelstartfunction)
EM.MaxStorageLevelFunction = pyo.Constraint(EM.Year,EM.Storage,EM.Hour,EM.Fuel, rule=maxstoragelevelfunction)
EM.StorageCostFunction = pyo.Constraint(EM.Year, EM.Storage, rule=storagecostfunction)
EM.StorageAnnualBalanceFunction = pyo.Constraint(EM.Year,EM.Storage,EM.Fuel, rule=storageannualbalancefunction)
EM.StorageMaximumCapacityConstraint = pyo.Constraint(EM.Year, EM.Storage, rule=storagemaxcapacityconstraint)
EM.TotalCapacityStorageAccounting = pyo.Constraint(EM.Year, EM.Storage, EM.Fuel, rule=totalstoragecapacityaccounting)
EM.GridCapacityFunction = pyo.Constraint(EM.Year,EM.Hour, rule=gridcapacity)

#for economics
EM.SalvageValueAccountingEndOfLife = pyo.Constraint(EM.Year,EM.Technology,rule=salvagevalue_endoflife)
EM.SalvageValueAccountingDuringLife = pyo.Constraint(EM.Year,EM.Technology,rule=salvagevalue_duringlife)


#### Model Instance

In [10]:
#creating the instance of the model here
instance = EM.create_instance()


#### Solver

In [11]:
#defining the solver here
opt = pyo.SolverFactory('gurobi')

#solving the model through the instance of the model
opt.solve(instance)

{'Problem': [{'Name': 'x5019768', 'Lower bound': 374127.88909901725, 'Upper bound': 374127.8890994586, 'Number of objectives': 1, 'Number of constraints': 5300079, 'Number of variables': 5019768, 'Number of binary variables': 0, 'Number of integer variables': 33, 'Number of continuous variables': 5019735, 'Number of nonzeros': 15914709, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '4786.02799987793', 'Error rc': 0, 'Time': 4822.45966219902}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### Solution and Results

In [12]:
#displaying results
#caution!!! do not uncomment below if running the code for multi-year analysis with a high resolution dataset, instead look at the results through generated csv's and tableau visualization
#instance.display()

#### Export Results

In [17]:
#extracting the data from the solution to export results

#producing a list of generation technologies of interest-- for filtering
technologies_list = ['SolarPV','WindOnshore','HydroPower','DieselGenerator','Tidal']

#we need the demand profiles of all fuels, even those where we did not give a default value
demandprofile = pd.Series(instance.DemandProfile.extract_values(),name='value',index=pd.MultiIndex.from_tuples(instance.DemandProfile.extract_values().keys(),names=['fuel', 'hour']))

#generating data frames out of our results
ProductionByTechnology = pd.DataFrame(instance.FuelProductionByTechnology.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.FuelProductionByTechnology.extract_values().keys(), names=('Year','Hour','Technology','Fuel')), columns=['Value']).reset_index()
NewCapacity = pd.DataFrame(instance.NewCapacity.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.NewCapacity.extract_values().keys(), names=('Year','Technology')), columns=['Value']).reset_index()
NewCapacity['Type'] = 'NewCapacity'
TotalCapacity = pd.DataFrame(instance.TotalCapacity.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.TotalCapacity.extract_values().keys(), names=('Year','Technology')), columns=['Value']).reset_index()
TotalCapacity['Type'] = 'TotalCapacity'
Demand = pd.DataFrame(readin_demand * demandprofile *(-1), columns=['value']).reset_index().rename(columns={'year':'Year', 'fuel': 'Fuel', 'hour': 'Hour', 'value' : 'Value'})
Use = pd.DataFrame(instance.FuelUseByTechnology.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.FuelUseByTechnology.extract_values().keys(), names=('Year','Hour','Technology','Fuel')), columns=['Value']).reset_index()
Curtailment = pd.DataFrame(instance.Curtailment.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.Curtailment.extract_values().keys(), names=('Year','Hour','Fuel')), columns=['Value']).reset_index()
StorageLevel = pd.DataFrame(instance.StorageLevel.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.StorageLevel.extract_values().keys(), names=('Year','Hour','Storage','Fuel')), columns=['Value']).reset_index().fillna(0)
NewStorageEnergyCapacity = pd.DataFrame(instance.NewStorageEnergyCapacity.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.NewStorageEnergyCapacity.extract_values().keys(), names=('Year','Storage', 'Fuel')), columns=['Value']).reset_index()
NewStorageEnergyCapacity['Type'] = 'NewCapacity'
TotalStorageCapacity = pd.DataFrame(instance.TotalStorageCapacity.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.TotalStorageCapacity.extract_values().keys(), names=('Year','Storage','Fuel')), columns=['Value']).reset_index()
TotalStorageCapacity['Type'] = 'TotalCapacity'
TotalCapitalCost = pd.DataFrame(instance.TotalCapitalCost.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.TotalCapitalCost.extract_values().keys(), names=('Year','Technology')), columns=['Value']).reset_index()
TotalStorageCost = pd.DataFrame(instance.TotalStorageCost.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.TotalStorageCost.extract_values().keys(), names=('Year','Storage')), columns=['Value']).reset_index()
TotalVariableCost = pd.DataFrame(instance.TotalVariableCost.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.TotalVariableCost.extract_values().keys(), names=('Year','Hour','Technology')), columns=['Value']).reset_index()
#TotalCapitalCost = (pd.DataFrame(instance.TotalCapitalCost.extract_values().values(), index=(instance.TotalCapitalCost.extract_values().keys()), columns=['Value']).rename_axis('Technology').reset_index()).iloc[0:5,:]
#TotalVariableCost = pd.DataFrame(instance.TotalVariableCost.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.TotalVariableCost.extract_values().keys(), names=('Hour', 'Technology')), columns=['Value']).reset_index()

#adding outputs for our storage flows
StorageCharge = pd.DataFrame(instance.StorageCharge.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.StorageCharge.extract_values().keys(), names=('Year','Hour','Technology','Fuel')), columns=['Value']).reset_index().fillna(0)
StorageDisCharge = pd.DataFrame(instance.StorageDisCharge.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.StorageDisCharge.extract_values().keys(), names=('Year','Hour','Technology','Fuel')), columns=['Value']).reset_index().fillna(0)

#making sure that the values for Curtailment, Use, StorageCharge are negative for our outputs
Use['Value']= Use['Value']*-1
Curtailment['Value']= Curtailment['Value']*-1
StorageCharge['Value']= StorageCharge['Value']*-1

#combining all values for Production, Use, Demand, and Curtailment into one data frame
EnergyBalance = pd.concat([ProductionByTechnology,Use], ignore_index=True)
EnergyBalance = pd.concat([EnergyBalance,Demand], ignore_index=True)
EnergyBalance['Technology'] = EnergyBalance['Technology'].fillna('Demand')
EnergyBalance = pd.concat([EnergyBalance,Curtailment], ignore_index=True)
EnergyBalance['Technology'] = EnergyBalance['Technology'].fillna('Curtailment')
EnergyBalance = pd.concat([EnergyBalance,StorageCharge], ignore_index=True)
EnergyBalance = pd.concat([EnergyBalance,StorageDisCharge], ignore_index=True)

#since we now have multiple capacity outputs, we have to do the same for capacities
Capacity = pd.DataFrame(columns=['Year', 'Technology', 'Type', 'Value'])
Capacity = pd.concat([Capacity,NewCapacity], ignore_index=True)
Capacity = pd.concat([Capacity,TotalCapacity], ignore_index=True)

#since we now have multiple storage capacity outputs, we have to do the same for storage capacities, also adjusting storage capacities again for dod. 
StorageCapacity = pd.DataFrame(columns=['Year', 'Storage','Fuel','Type', 'Value'])
StorageCapacity = pd.concat([StorageCapacity,NewStorageEnergyCapacity], ignore_index=True)
StorageCapacity = pd.concat([StorageCapacity,TotalStorageCapacity], ignore_index=True)
def adjstoragecapacity(row):
    if row[1] == 'LeadAcidBattery':
        return row[4] / 0.4
    elif row[1] == 'LFPBattery':
        return row[4] / 0.8
    else:
        return row[4]
StorageCapacity.iloc[:, 4] = StorageCapacity.apply(adjstoragecapacity, axis=1)

#now, information on emissions would also be interesting
AnnualTechnologyEmissions = pd.DataFrame(instance.AnnualTechnologyEmissions.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.AnnualTechnologyEmissions.extract_values().keys(), names=('Year','Technology')), columns=['Value']).reset_index() 
def adjtechnologyemissions(x):
    AnnualTechnologyEmissions.iloc[[7, 18, 29], 2] = [0.65 * x, 0.30 * x, 0.00 * x]
    AnnualTechnologyEmissions.iloc[[8, 19, 30], 2] = [0.7 * 0.25 * x, 0.5 * 0.25 * x, 0.00 * x]
    AnnualTechnologyEmissions.iloc[[9,20,31],2] *= 0.4
    AnnualTechnologyEmissions.iloc[20, 2] = AnnualTechnologyEmissions.iloc[9, 2] * 0.55
    AnnualTechnologyEmissions.iloc[31, 2] = AnnualTechnologyEmissions.iloc[9, 2] * 0.11
    return AnnualTechnologyEmissions
adjtechnologyemissions(400)

#getting investment profile across the years for technologies and storages
TotalCapitalCost = TotalCapitalCost[TotalCapitalCost['Technology'].isin(technologies_list)]
TotalStorageCost.rename(columns={'Storage':'Technology'}, inplace=True)
TotalAnnualCapitalCosts = pd.concat([TotalCapitalCost,TotalStorageCost], axis=0).sort_values(by=['Year','Technology'], ascending=True).reset_index().drop('index', axis=1)

#getting variable costs hourly profile for each year
TotalVariableCost = TotalVariableCost[TotalVariableCost['Technology'].isin(technologies_list)]

#getting annual curtailment profiles
PowerCurtailment = Curtailment[Curtailment['Fuel'] == 'Power']
YearlyPowerCurtailment = PowerCurtailment.drop(columns=['Fuel','Hour'], axis=1).groupby(['Year']).sum()

#Variablecosts --total and hourlyaverages in $/kWh of energy generation
HourlyVariableCost = TotalVariableCost.drop(columns=['Technology'], axis=1).groupby(['Year','Hour']).sum()
ProductionforVariableCost = ProductionByTechnology[ProductionByTechnology['Technology'].isin(technologies_list)]
SumProductionforVariableCost = ProductionforVariableCost.drop(columns=['Technology','Fuel'], axis=1).groupby(['Year','Hour']).sum()
AnnualTotalVariableCosts = HourlyVariableCost.droplevel(1, axis=0).groupby(['Year']).sum()
AverageVariableCostTimeseries = HourlyVariableCost['Value']/SumProductionforVariableCost['Value']

#writing our output values to csv files in the results subfolder
EnergyBalance.to_csv('../3. Output/EnergyBalance.csv',index=False)
Capacity.to_csv('../3. Output/Capacity.csv',index=False)
StorageLevel.to_csv('../3. Output/StorageLevel.csv',index=False)
AnnualTechnologyEmissions.to_csv('../3. Output/Emissions.csv',index=False)
Curtailment.to_csv('../3. Output/Curtailment.csv',index=False)
StorageCapacity.to_csv('../3. Output/StorageCapacity.csv', index=False)
TotalAnnualCapitalCosts.to_csv('../3. Output/TotalAnnualCapacityCosts.csv', index=False)
YearlyPowerCurtailment.to_csv('../3. Output/YearlyPowerCurtailment.csv',index=True)
AnnualTotalVariableCosts.to_csv('../3. Output/AnnualTotalVariableCosts.csv', index=True)
AverageVariableCostTimeseries.to_csv('../3. Output/AverageVariableCostTimeseries.csv', index=True)

print(sum(instance.AnnualTechnologyEmissions.extract_values().values()))
print(sum(instance.Curtailment.extract_values().values()))
data = instance.InfeasibleGeneration.extract_values()
non_zero_values = {key: value for key, value in data.items() if value != 0.0}
non_zero_values
sum(instance.InfeasibleGeneration.extract_values().values())
print(max(instance.InfeasibleGeneration.extract_values().values()))


4364.804841255169
135183.6763785716
0.0


In [14]:
#seeing the proportion of heat load served between electricheatpump and wood/conventionalfuelburners
Heatloadserved = ProductionByTechnology[ProductionByTechnology['Technology'].isin(['ElectricHeatPump','WoodBurner'])]
TotalHeatLoad = Heatloadserved.groupby(['Year','Technology'])['Value'].sum()
print(TotalHeatLoad)

Year  Technology      
2026  ElectricHeatPump    198213.084290
      WoodBurner          882060.820310
2028  ElectricHeatPump    180034.398840
      WoodBurner          940091.431060
2030  ElectricHeatPump    564899.143111
      WoodBurner          595386.750689
Name: Value, dtype: float64


In [15]:
'''
#scrap yard


def curtailmentdischargelimit(EM,h):
    return EM.CurtailmentState[h] + EM.BatteryChargeState[h] <= 1

def curtailmentstate(EM,h):
    if (EM.Curtailment[y,f,h] for y in EM.Year for f in EM.Fuel) > 0:
        return EM.CurtailmentState[h]  == 1
    else:
        return EM.CurtailmentState[h]  == 0

def batterychargestate(EM,h):
    if (EM.StorageDischarge[y,s,h,f] for y in EM.Year for s in EM.Storage for f in EM.Fuel) > 0:
        return EM.BatteryChargeState[h] == 1
    else:
        return EM.BatteryChargeState[h] == 0

def totalcapacity(EM,y,t):
    return EM.TotalCapacity[y,t] == EM.ResidualCapacity[t] + EM.NewCapacity[y,t]
        

def totalcapitalcostfunction(EM,t):
    return sum(EM.NewCapacity[y,t] * EM.CapitalCost[t] / (1+EM.DiscountRate)**(y-min(EM.Year)) for y in EM.Year) == EM.TotalCapitalCost[t]

def totalvariablecostfunction(EM,h,t):
    return sum(EM.FuelProductionByTechnology[y,h,t,f] / (1+EM.DiscountRate)**(y-min(EM.Year))
            * EM.VariableCost[t] * EM.YearlyDifferenceMultiplier[y] for f in EM.Fuel for y in EM.Year) == EM.TotalVariableCost[h,t]

def totalstoragecapacity(EM,s,f):
    if EM.StorageDisChargeEfficiency[s,f] > 0:
        return EM.TotalStorageCapacity[s,f] == EM.ResidualCapacityStorage[s,f] + EM.StorageEnergyCapacity[s,f]
    else:
        return pyo.Constraint.Skip

def batterySOCfunction(EM,s,h,f,):
    if EM.StorageDisChargeEfficiency[s,f] > 0:
        return EM.StorageLevel[h,s,f] >= 0.6 * EM.StorageEnergyCapacity[s,f]
    else:
        return pyo.Constraint.Skip

def totalemissionaccounting(EM):
    return (
        sum(EM.AnnualTechnologyEmissions[y,t] * EM.YearlyDifferenceMultiplier[y] for y in EM.Year for t in EM.Technology) == EM.ModelPeriodEmissions
    )

def totalemissionlimit(EM):
    return (
        EM.ModelPeriodEmissionLimit >= EM.ModelPeriodEmissions
    )        
'''

'''
#useful scrap
ProductionByTechnology = pd.DataFrame(instance.FuelProductionByTechnology.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.FuelProductionByTechnology.extract_values().keys(), names=('Hour','Technology', 'Fuel')), columns=['Value']).reset_index()
Capacity = pd.DataFrame(instance.Capacity.extract_values().values(), index=(instance.Capacity.extract_values().keys()), columns=['Value']).rename_axis('Technology').reset_index()
Demand = pd.DataFrame(readin_demand * demandprofile *(-1), columns=['value']).reset_index().rename(columns={'fuel': 'Fuel', 'hour': 'Hour', 'value' : 'Value'})
Use = pd.DataFrame(instance.FuelUseByTechnology.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.FuelUseByTechnology.extract_values().keys(), names=('Hour','Technology', 'Fuel')), columns=['Value']).reset_index()
Curtailment = pd.DataFrame(instance.Curtailment.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.Curtailment.extract_values().keys(), names=('Hour','Fuel')), columns=['Value']).reset_index()
StorageLevel = pd.DataFrame(instance.StorageLevel.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.StorageLevel.extract_values().keys(), names=('Hour','Storage','Fuel')), columns=['Value']).reset_index().fillna(0)
TotalCapacity = pd.DataFrame(instance.TotalCapacity.extract_values().values(), index=(instance.TotalCapacity.extract_values().keys()), columns=['Value']).rename_axis('Technology').reset_index()
TotalStorageCapacity = (pd.DataFrame(instance.TotalStorageCapacity.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.TotalStorageCapacity.extract_values().keys(), names=('Storage', 'Fuel')), columns=['Value']).reset_index().dropna()).drop('Fuel', axis=1).reset_index()
GenerationCapacity = TotalCapacity.iloc[0:5,:]
TotalCapitalCost = (pd.DataFrame(instance.TotalCapitalCost.extract_values().values(), index=(instance.TotalCapitalCost.extract_values().keys()), columns=['Value']).rename_axis('Technology').reset_index()).iloc[0:5,:]
TotalVariableCost = pd.DataFrame(instance.TotalVariableCost.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.TotalVariableCost.extract_values().keys(), names=('Hour', 'Technology')), columns=['Value']).reset_index()
TotalStorageCost = pd.DataFrame(instance.TotalStorageCost.extract_values().values(), index=(instance.TotalStorageCost.extract_values().keys()), columns=['Value']).rename_axis('Technology').reset_index()
TotalProjectCapital = pd.concat([TotalCapitalCost, TotalStorageCost], ignore_index=True)
TechnologyEmissions = pd.DataFrame(instance.TechnologyEmissions.extract_values().values(), index=(instance.TechnologyEmissions.extract_values().keys()), columns=['Value']).rename_axis('Technology').reset_index()

#let's add outputs for our storage flows
StorageCharge = pd.DataFrame(instance.StorageCharge.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.StorageCharge.extract_values().keys(), names=('Hour','Technology','Fuel')), columns=['Value']).reset_index().fillna(0)
StorageDisCharge = (pd.DataFrame(instance.StorageDisCharge.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.StorageDisCharge.extract_values().keys(), names=('Hour','Technology','Fuel')), columns=['Value']).reset_index().fillna(0))

# make sure that the values for Curtailment and Use are negative for our outputs
Use['Value']= Use['Value']*-1
Curtailment['Value']= Curtailment['Value']*-1
StorageCharge['Value']= StorageCharge['Value']*-1

# combine all values for Production, Use, Demand, and Curtailment into one data frame
EnergyBalance = pd.concat([ProductionByTechnology,Use], ignore_index=True)
EnergyBalance = pd.concat([EnergyBalance,Demand], ignore_index=True)
EnergyBalance['Technology'] = EnergyBalance['Technology'].fillna('Demand')
EnergyBalance = pd.concat([EnergyBalance,Curtailment], ignore_index=True)
EnergyBalance['Technology'] = EnergyBalance['Technology'].fillna('Curtailment')
EnergyBalance = pd.concat([EnergyBalance,StorageCharge], ignore_index=True)
EnergyBalance = pd.concat([EnergyBalance,StorageDisCharge], ignore_index=True)

# write our output values to csv files in the output subfolder
EnergyBalance.to_csv('../3. Output/EnergyBalance.csv',index=False)
Capacity.to_csv('../3. Output/Capacity.csv',index=False)
StorageLevel.to_csv('../3. Output/StorageLevel.csv',index=False)
TotalStorageCapacity.to_csv('../3. Output/TotalStorageCapacity.csv', index=False)
GenerationCapacity.to_csv('../3. Output/GenerationCapacity.csv', index=False)
TotalProjectCapital.to_csv('../3. Output/TotalProjectCapital.csv', index=False)
TotalVariableCost.to_csv('../3. Output/TotalVariableCost.csv', index=False)
TechnologyEmissions.to_csv('../3. Output/TechnologyEmissions.csv', index=False)

#print(TotalCapacity)
#print(TotalStorageCapacity)
#print(GenerationCapacity)
#print(TotalCapitalCost)
#print(TotalProjectCapital)
print(TechnologyEmissions)

'''

"\n#useful scrap\nProductionByTechnology = pd.DataFrame(instance.FuelProductionByTechnology.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.FuelProductionByTechnology.extract_values().keys(), names=('Hour','Technology', 'Fuel')), columns=['Value']).reset_index()\nCapacity = pd.DataFrame(instance.Capacity.extract_values().values(), index=(instance.Capacity.extract_values().keys()), columns=['Value']).rename_axis('Technology').reset_index()\nDemand = pd.DataFrame(readin_demand * demandprofile *(-1), columns=['value']).reset_index().rename(columns={'fuel': 'Fuel', 'hour': 'Hour', 'value' : 'Value'})\nUse = pd.DataFrame(instance.FuelUseByTechnology.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.FuelUseByTechnology.extract_values().keys(), names=('Hour','Technology', 'Fuel')), columns=['Value']).reset_index()\nCurtailment = pd.DataFrame(instance.Curtailment.extract_values().values(), index=pd.MultiIndex.from_tuples(instance.Curtailment.extract_values